### Pregunta de Negocio
¿Cómo optimizamos la satisfacción y retención de usuarios mediante recomendaciones musicales personalizadas con base en el estado del ánimo del usuario? 

La motivación detrás de este proyecto surge de la sobrecarga de opciones musicales en la era digital debido al acceso a un vasto catálogo de música, que a veces puede resultar abrumador para los usuarios. En muchas ocasiones, los usuarios se sienten perdidos entre la multitud de opciones, lo que conduce a una experiencia despersonalizada y, en última instancia, a la pérdida de interés. Además, reconocemos la importancia de la emoción y la conexión en la música, ya que sabemos que la música tiene el poder de influir en nuestras emociones y estados de ánimo. Nuestra meta es aprovechar esta capacidad para crear una experiencia musical que no solo sea placentera, sino que también establezca una conexión emocional profunda entre los usuarios y la música que escuchan. 

Los clientes potenciales de este proyecto son los usuarios de servicios de streaming de música que desean compartir su estado de ánimo para recibir recomendaciones más personalizadas. El contexto organizacional donde surge este problema es el ámbito de las empresas que ofrecen este tipo de servicios. Estas plataformas buscan continuamente mejorar su servicio con el fin de satisfacer y retener a la mayor cantidad posible de usuarios. 

 

Para resolver el problema, utilizaremos un algoritmo híbrido de recomendación que combinará filtros colaborativos y basados en contenido para generar una lista de recomendaciones. Por ejemplo, podríamos utilizar la recomendación colaborativa para seleccionar un conjunto inicial de elementos y luego refinar esa lista utilizando el filtrado basado en contenido. Con esto, optimizaremos la satisfacción de los usuarios y aumentaremos su retención. 

### 1. Lectura Dataset

In [4]:
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("./data/data.csv")
df_gener = pd.read_csv("./data/data_by_genres.csv")
df_gener_arts = pd.read_csv("./data/data_w_genres.csv")

In [4]:
print(df.shape)
df.head()

(170653, 19)


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


La información recolactada es acerca de diferentes caracteristicas de canciones y artistas para generar un sistema de recomendación para la plataforma Spotify.

Cada registro contiene 19 caracteristicas, las variables son:

0. valence: Una medida de la positividad de una canción. Valores más altos indican canciones más alegres.
1. year: El año de lanzamiento de la canción.
2. acousticness: Una medida de cuán acústica es una canción. Valores cercanos a 1 indican alta acústica.
3. artists: El nombre de los artistas que realizaron la canción.
4. danceability: Una medida de cuán adecuada es una canción para bailar.
5. duration_ms: La duración de la canción en milisegundos.
6. energy: Una medida de la energía de la canción. Valores más altos indican canciones más enérgicas.
7. explicit: Un indicador binario que indica si la canción contiene contenido explícito (1) o no (0).
8. id: Un identificador único para la canción.
9. instrumentalness: Una medida de cuánta presencia de voz o palabras tiene la canción. Valores más altos indican canciones instrumentales.
10. key: La tonalidad de la canción.
11. liveness: Una medida de cuán en vivo suena una grabación. Valores más altos indican que suena más en vivo.
12. loudness: Una medida del volumen general de la canción.
13. mode: La modalidad de la canción (mayor o menor).
14. name: El nombre de la canción.
15. popularity: La popularidad de la canción.
16. release_date: La fecha de lanzamiento de la canción.
17. speechiness: Una medida de cuán "hablada" es una canción en lugar de cantada.
18. tempo: El tempo de la canción (ritmo o velocidad).

Nuestro set de datos tiene un poco más de 170k registros y el total de 19 variables que mencionamos anteriormente

In [5]:
print("Existen {} artistas unicos en nuestro dataset".format(len(df['artists'].unique())))

Existen 34088 artistas unicos en nuestro dataset
